<center>Progetto realizzato da Elena Curti (matr. 185431)

# Recipes
</center>

## Requisiti
Questo progetto è stato realizzato con Python 3.11 e Neo4j. E' necessaria l'installazione dei seguenti pacchetti:

In [1]:
# %pip install py2neo neo4j-driver

## Operazioni iniziali
Per il corretto funzionamento del progetto proposto, è necessario:
- Creare un nuovo progetto Neo4j 
- Creare all'interno del progetto un DMBS (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0)
- Aggiungere il plugin APOC.
- Premere su Start.
<!-- 
DBMS e importarci i dati, seguendo i seguenti passi. <br>
Da Neo4j creare un nuovo progetto. Scegliere una tra le seguenti alternative:
- Dal progetto appena creato, aggiungere al progetto il file [neo4j.dump](data/neo4j.dump). Creare poi un nuovo DBMS dal file dump (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0). Aggiungere il plugin APOC. Premere su Start.
- Dal progetto appena creato, aggiungere un DBMS locale (scegliendo come nome, password e versione i valori indicati al punto sopra). Aggiungere il plugin APOC. Premere su Start e poi su Open. Eseguire tutti i comandi riportati nel file [import_commands.txt](data/import_commands.txt).
<br>-->
Eseguire poi le celle di codice, nell'ordine proposto 

## Connessione e autenticazione

In [1]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "pass"))

print("Connessione al database riuscita!")

#  \nEcco 5 ricette d'esempio:")
# cq = "MATCH (p:Recipe) RETURN p.name LIMIT 5"
# for rec in graph.run(cq):
    # print("\t",str(rec)[1:-1])

Connessione al database riuscita!


# Schema iniziale
Lo use case originale è stato trovato sul sito [Recipes Neo4j](https://neo4j.com/graphgists/dd3dedcf-c377-4575-84f4-4d0d30b2a4c5/). Ha la seguente struttura iniziale:
```
    call db.schema.visualization()
```

![schema](img/db_schema2.png)

I nodi Ingredient, Author, Collection, DietType, Keyword ha un solo campo "name" che rappresenta il nome dell'ingrediente. <br>
Il nodo Recipe ha le seguenti proprietà:
<ul>
    <li>id: campo che identifica univocamente la ricetta </li>
    <li>cookingTime: tempo di cottura </li>
    <li>description: descrizione della ricetta </li>
    <li>name: nome della ricetta </li>
    <li>preparationTime: tempo di preparazione </li>
    <li>skillLevel: difficoltà </li>
</ul>
Gli archi non hanno proprietà.

## Import dei dati

In [9]:
print("Inserimento dei dati in corso... ")

# Ricette
graph.run(""" 
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.title AS title,
       value.page.article.description AS description,
       value.page.recipe.cooking_time AS cookingTime,
       value.page.recipe.prep_time AS preparationTime,
       value.page.recipe.skill_level AS skillLevel
MERGE (r:Recipe {id: id})
SET r.cookingTime = cookingTime,
    r.preparationTime = preparationTime,
    r.name = title,
    r.description = description,
    r.skillLevel = skillLevel;
""")
graph.run(""" CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(name)  """)

# Autori
graph.run("""
//import authors and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.article.author AS author
MERGE (a:Author {name: author})
WITH a,id
MATCH (r:Recipe {id:id})
MERGE (a)-[:WROTE]->(r);
""")
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);""" )

# Ingredienti
graph.run("""//import ingredients and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.ingredients AS ingredients
MATCH (r:Recipe {id:id})
FOREACH (ingredient IN ingredients |
  MERGE (i:Ingredient {name: ingredient})
  MERGE (r)-[:CONTAINS_INGREDIENT]->(i)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);""" )

# DietType
graph.run("""//import dietTypes and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.diet_types AS dietTypes
MATCH (r:Recipe {id:id})
FOREACH (dietType IN dietTypes |
  MERGE (d:DietType {name: dietType})
  MERGE (r)-[:DIET_TYPE]->(d)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);""" )

# Collections
graph.run("""//import collections and connect to recipes
CALL apoc.load.json('https://raw.githubusercontent.com/neo4j-examples/graphgists/master/browser-guides/data/stream_clean.json') YIELD value
WITH value.page.article.id AS id,
       value.page.recipe.collections AS collections
MATCH (r:Recipe {id:id})
FOREACH (collection IN collections |
  MERGE (c:Collection {name: collection})
  MERGE (r)-[:COLLECTION]->(c)
);""" )
graph.run("""CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);""" )

print("Tutti i dati sono stati importati correttamente!")

Inserimento dei dati in corso... 
Tutti i dati sono stati importati correttamente!


## Aggiunta degli indici
Per migliorare la performance delle cypher query sono stati aggiunti i seguenti indici: 
    <ul>
        <li><i>id</i> su Recipe</li>
        <li><i>name</i> su Ingredient</li>
        <li><i>name</i> su Keyword</li>
        <li><i>name</i> su DietType</li>
        <li><i>name</i> su Author</li>
        <li><i>name</i> su Collection</li>
    </ul>
Per creare gli indici è stato usato il seguente script. Il comando ```CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id)``` è equivalente a ```CREATE INDEX IF NOT EXISTS ON :Recipe(name)```. Il primo, però, è compatibile con la versione 5.2.0 di Neo4j.

In [6]:
def run_multiple_query(multiple_query):
    for query in multiple_query.splitlines():
        if query == "": continue
        graph.run(query)

cq_versione5= """
CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(name) 
CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);
""" 
run_multiple_query(cq_versione5)

## Aggiunta di dati e modifiche allo schema originale
Per arricchire il progetto ho deciso di aggiungere dei dati presi da Kaggle e di eseguire le seguenti modifiche allo schema originale. 
<br>
[OK] Aggiunto il campo is_allergene negli ingredienti. (TODO andrà messo dopo)<br>
[ok] Rimosse le keyword<br>
Aggiunte le altre ricette e la proprieta persone_pezzi, link, bbc_or_gf nella ricetta<br>
aggiunto campo quantita in contains_ingredients<br>
aggiunti altri campi dal file originale  <br>

<MARK> TODO ARRIVATA QUI  -> Aggiungere https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes

### 1. Copia dei file 
Prima di effettuare le modifiche, copio i file necessari nella cartella "import" del DBMS, mediante il seguente script:

In [2]:
import os 
import shutil

DATA_FOLDER ="data"+os.path.sep
cq ="""CALL dbms.listConfig() YIELD name, value
WHERE name="server.directories.import"
RETURN value"""
dir_import = graph.run(cq).evaluate()
print("Cartella import del DMBS: " + dir_import)

for file_name in ["FoodData.csv", "gz_recipe.csv"]:#os.listdir(DATA_FOLDER):
    source = DATA_FOLDER + file_name
    destination = dir_import + os.path.sep + file_name
    shutil.copy(source, destination)



Cartella import del DMBS: D:\Elena\_Elena\Shared\Universita\Magistrale\Big_data\Anno_22_23\TO_DEL_QUANDO_ESAME_DATO\Neo4J\relate-data\dbmss\dbms-b3e0a01c-d1be-4257-8d27-97054bc0e187\import


### 2. Rimuovo il nodo Keyword
Ho deciso di rimuovere il nodo keyword, mediante il seguente script:

In [4]:
cq="""
MATCH (n:Keyword) DETACH DELETE n
"""
graph.run(cq)

(No data)

### 3. Aggiunta di altre ricette
Per arricchire il dataset, ho deciso di inserire altre ricette al database. Ho quindi scaricato il file ([gz_recipe.csv](data/gz_recipe.csv)) da https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes, contenente delle ricette estratte dal sito [GialloZafferano](https://www.giallozafferano.it/). <br>
Il file scaricato contiene i seguenti campi:
- Nome: nome del piatto
- Categoria: categoria della ricetta (Primi, Secondi, ...)
- Link: link della ricetta
- Persone/Pezzi: per quante persone è la "calibrata" la ricetta/numero di pezzi
- Ingredienti: lista di ingredienti e relative quantità. Ad esempio: [['Mascarpone', '750g'], ['Uova', '260g']]
- Steps: contenuto della ricetta
Per i inserire questi dati ho deciso di:
- Aggiungere a Recipe le proprietà 
    - "fonte" che avrà il valore "GialloZafferano" o "BBC GoodFood"
    - "persone_pezzi" per indicare il numero di persone o pezzi
- Aggiungere a CONTAINS_INGREDIENTS una proprietà "quantita"
- Creare un nodo Collection per ogni categoria


(No data)

In [9]:
class MiaStopExecution(Exception):
    """Il raise di questa classe provoca l'interruzione dell'esecizione della cella, senza interrompere il kernel"""
    def _render_traceback_(self):
        pass

In [10]:
# Aggiungo le ricette di GialloZafferano. 

# Controllo che non ci siano ricette già presenti nel database con lo stesso id. 
# Se questa cella di codice viene eseguita più di una volta, le ricette non vengono reinserite
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MATCH (r:Recipe {id:value["id"]})
return count(r)
"""
if graph.run(cq).evaluate() != 0:
    print("Le ricette sono gia' state inserite!")
    raise(MiaStopExecution)

# Inserisco le ricette
cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
MERGE (r:Recipe {id: value["id"]})
SET r.name = value["Nome"],
    r.description = value["Steps"],
    r.persone_pezzi = value["Persone/Pezzi"],
    r.fonte = "GialloZafferano"
"""
graph.run(cq)

# Inserisco gli ingredienti

cq="""
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
WITH value["id"] AS id,
WITH replace(value["Ingredienti"], "[[", "") AS ris1
WITH replace(ris1, "]]", "") AS ris2
WITH split(ris2, "], [") AS ingr_list

MATCH (r:Recipe {id:id})
FOREACH (ingredient IN ingredients |
  MERGE (i:Ingredient {name: ingredient})
  MERGE (r)-[:CONTAINS_INGREDIENT]->(i)
);
""" 


# Creo le categorie 
cq="""LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
CREATE (n:Collection {name: value["Categoria"]})
"""



Le ricette sono gia' state inserite


In [ ]:
cq=""" 
LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
//MATCH (r:Recipe )
return value["id"] as id
ORDER BY id 
"""
for i in graph.run(cq):
    print(i)

In [ ]:
cq="""LOAD CSV WITH HEADERS FROM 'file:///gz_recipe.csv' AS value
WITH replace(value["Ingredienti"], "[[", "") AS ris1
WITH replace(ris1, "]]", "") AS ris2
WITH split(ris2, "], [") AS ingr_list
RETURN ingr_list
"""
tmp = graph.run(cq)
for i in graph.run(cq):
    print(i)
    print()

### 1. Ingredienti
Ho aggiunto un campo "is_allergene" al nodo Ingredient. Ho inizialmente scaricato da https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies un file ([FoodData.csv](data/FoodData.csv)) con l'elenco degli allergeni. Ho quindi impostato gli ingredienti contenuti nel file con is_allergene=True. Ho invece settato is_allergene=False per i restanti ingredienti. 

In [4]:
# Setto gli allergeni
cq="""LOAD CSV WITH HEADERS FROM 'file:///FoodData.csv' AS value
MATCH (i:Ingredient)
WHERE toLower(i.name) CONTAINS toLower(value["Food"]) 
SET i.is_allergene=True;"""
graph.run(cq)

# Setto i restanti ingredienti come non allergeni
cq="""
MATCH (i:Ingredient)
WHERE i.is_allergene IS NULL
SET i.is_allergene=False; """
graph.run(cq)


(No data)

(No data)

In [7]:
cq = "MATCH (d:Recipe) RETURN d"
# print(cq)
# for i in graph.run(cq):
#     print(i)